# A* search for path finding

In [ ]:

import heapq
class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0  # Cost from start to node
        self.h = 0  # Heuristic cost to goal
        self.f = 0  # Total cost

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    # Manhattan distance
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def astar(grid, start, end):
    open_list = []
    closed_set = set()

    start_node = Node(start)
    end_node = Node(end)

    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)
        closed_set.add(current_node.position)

        if current_node.position == end_node.position:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        (x, y) = current_node.position
        neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]

        for next_pos in neighbors:
            (nx, ny) = next_pos
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == 0:
                if next_pos in closed_set:
                    continue

                neighbor = Node(next_pos, current_node)
                neighbor.g = current_node.g + 1
                neighbor.h = heuristic(neighbor.position, end_node.position)
                neighbor.f = neighbor.g + neighbor.h

                if all(open_node.position != neighbor.position or open_node.f > neighbor.f for open_node in open_list):
                    heapq.heappush(open_list, neighbor)

    return None  # No path found

# Example usage
grid = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
end = (4, 4)
path = astar(grid, start, end)
print("Path:", path)


# A* search for 8 puzzle solve

In [ ]:
import heapq

goal_state = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5]
]

def state_to_tuple(state):
    return tuple(num for row in state for num in row)

def misplaced_tiles(state):
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                count += 1
    return count

def get_neighbors(state):
    neighbors = []
    x, y = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = [row[:] for row in state]
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append(new_state)
    return neighbors

def print_state(state):
    for row in state:
        print(row)
    print()

def a_star_verbose(start_state):
    open_list = []
    count = 0  # Tie-breaker for equal f values
    heapq.heappush(open_list, (0, count, start_state))
    came_from = {}
    g_score = {state_to_tuple(start_state): 0}
    visited = set()

    step = 0
    while open_list:
        f, _, current = heapq.heappop(open_list)
        current_tuple = state_to_tuple(current)
        visited.add(current_tuple)

        g = g_score[current_tuple]
        h = misplaced_tiles(current)

        print(f"Step {step}: Expanding node with g(n)={g}, h(n)={h}, f(n)={f}")
        print_state(current)
        step += 1

        if current == goal_state:
            print("Reached goal state!\n")
            path = [current]
            while current_tuple in came_from:
                current = came_from[current_tuple]
                current_tuple = state_to_tuple(current)
                path.append(current)
            return path[::-1]

        for neighbor in get_neighbors(current):
            neighbor_tuple = state_to_tuple(neighbor)
            tentative_g = g + 1

            if neighbor_tuple in visited:
                continue

            if tentative_g < g_score.get(neighbor_tuple, float('inf')):
                g_score[neighbor_tuple] = tentative_g
                h_n = misplaced_tiles(neighbor)
                f_n = tentative_g + h_n
                count += 1
                heapq.heappush(open_list, (f_n, count, neighbor))
                came_from[neighbor_tuple] = current

                print(f"  Neighbor with g={tentative_g}, h={h_n}, f={f_n}:")
                print_state(neighbor)
    
    print("No solution found.")
    return None

# Example starting state
start = [
    [2, 8, 3],
    [1, 6, 4],
    [7, 0, 5]
]

solution = a_star_verbose(start)

if solution:
    print("✔ Final solution found in", len(solution) - 1, "moves:\n")
    for i, step in enumerate(solution):
        print(f"Step {i}:")
        print_state(step)
else:
    print("❌ No solution found.")


In [7]:
import heapq

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

def state_to_tuple(state):
    return tuple(num for row in state for num in row)

def misplaced_tiles(state):
    return sum(
        1 for i in range(3) for j in range(3)
        if state[i][j] != 0 and state[i][j] != goal_state[i][j]
    )

def get_neighbors(state):
    neighbors = []
    x, y = [(i, row.index(0)) for i, row in enumerate(state) if 0 in row][0]
    directions = [(-1,0), (1,0), (0,-1), (0,1)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = [row[:] for row in state]
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append(new_state)
    return neighbors

def print_state(state):
    for row in state:
        print(row)
    print()

def best_first_search(start_state):
    open_list = []
    heapq.heappush(open_list, (misplaced_tiles(start_state), start_state))
    visited = set()
    came_from = {}
    visited.add(state_to_tuple(start_state))

    steps = 0
    while open_list:
        h, current = heapq.heappop(open_list)

        print(f"Step {steps}: Expanding node with h(n) = {h}")
        print_state(current)
        steps += 1

        if current == goal_state:
            print("🎯 Goal state reached!\n")
            path = [current]
            while state_to_tuple(current) in came_from:
                current = came_from[state_to_tuple(current)]
                path.append(current)
            return path[::-1]

        for neighbor in get_neighbors(current):
            neighbor_tuple = state_to_tuple(neighbor)
            if neighbor_tuple not in visited:
                visited.add(neighbor_tuple)
                came_from[neighbor_tuple] = current
                h_n = misplaced_tiles(neighbor)
                heapq.heappush(open_list, (h_n, neighbor))

    print("❌ No solution found.")
    return None

# Example initial state
start_state = [
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
]

solution = best_first_search(start_state)

if solution:
    print("🧩 Solution path (may not be optimal):\n")
    for i, step in enumerate(solution):
        print(f"Move {i}:")
        print_state(step)
else:
    print("No path to goal found.")


Step 0: Expanding node with h(n) = 2
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Step 1: Expanding node with h(n) = 1
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step 2: Expanding node with h(n) = 0
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

🎯 Goal state reached!

🧩 Solution path (may not be optimal):

Move 0:
[1, 2, 3]
[4, 0, 6]
[7, 5, 8]

Move 1:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Move 2:
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

